In [1]:
import os 
import sys 
import pandas as pd 
import re
import pickle
import json
import sqlite3
from pprint import pprint
import datetime
import pdb

In [2]:
sys.version

'3.6.9 (default, Oct  8 2020, 12:12:24) \n[GCC 8.4.0]'

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

In [4]:
%load_ext autoreload
%autoreload 2

## Set up 

In [5]:
with open('../credentials/spotify_creds.pkl', 'rb') as hnd:
    credentials = pickle.load(hnd)

In [6]:
os.environ.update(credentials)

In [7]:
scope = "user-read-recently-played"

In [8]:
spotify = spotipy.Spotify(client_credentials_manager= SpotifyOAuth(scope=scope
                                                                   , username='malchemist02'))

## Functions

In [9]:
def split_utc_time_str(time_str):
    if '.' in time_str:
        part1 = datetime.datetime.strptime(time_str, '%Y-%m-%dT%H:%M:%S.%fZ') - datetime.timedelta(hours=4)
    else:
        part1 = datetime.datetime.strptime(time_str, '%Y-%m-%dT%H:%M:%SZ') - datetime.timedelta(hours=4)
    date = str(part1.date())
    time = str(part1.time().strftime('%H:%M'))
    return(date, time)

In [10]:
time1 = split_utc_time_str('2020-11-13T21:32:20Z')
time1

('2020-11-13', '17:32')

In [11]:
def get_recently_played(after=None):
    recently_played = spotify.current_user_recently_played(after=after)
    # Describe json 
    print('Getting songs at ', datetime.datetime.now())
    
    if recently_played['cursors'] == None:
        return(pd.DataFrame(columns = ['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id']))
    
    before = recently_played['cursors']['before']
    after = recently_played['cursors']['after']
    n_items = len(recently_played['items'])
    print(f'There are {n_items} songs between '
          , datetime.datetime.fromtimestamp(int(before)/1000)
          , '  and  '
          , datetime.datetime.fromtimestamp(int(after)/1000))
    
    if n_items == 0:
        return(pd.DataFrame(columns = ['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id']))
    
    # Parse recently played output 
    songs_list = []
    wanted_keys = ['name', 'duration_ms', 'popularity', 'uri']
    for song in recently_played['items']:
        temp_song_dict = {}
        temp_song_dict['after_ts'] = after
        temp_song_dict['played_at_date'], temp_song_dict['played_at_time'] = split_utc_time_str(song['played_at'])
        for k in wanted_keys:
            temp_song_dict[k] = song['track'][k]
            
        # Take first artist information 
        temp_song_dict['artist_name'] = song['track']['artists'][0]['name'] 
        temp_song_dict['artist_id'] = song['track']['artists'][0]['uri']

        if song['context'] != None:
            temp_song_dict['playlist_id'] = song['context']['uri']
        else:
            temp_song_dict['playlist_id'] = None
        songs_list.append(temp_song_dict)

    
    # Export cleanly 
    out_series = pd.DataFrame(songs_list)
    out_series['duration_min'] = out_series['duration_ms']/60000.0
    out_series = out_series.rename({'uri':'song_uri'}, axis=1) 
    out_series = out_series[['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id', 'after_ts']]
    
    return(out_series)

## Get time of last Spotify Pull

In [12]:
db_location = 'data/listening_history.db'

In [13]:
con = sqlite3.connect(db_location)

In [14]:
cursor = con.cursor()

In [15]:
try:
    rslt = cursor.execute('select max(after_ts) from Listening_History').fetchone()
    latest_time_pull = rslt[0]
    print("Latest pull from ", 
    datetime.datetime.fromtimestamp(int(latest_time_pull)/1000)) 
except Exception as e:
    # Mainly for first run through when table doesn't exist yet 
    ## Will take the latest recent history available 
    print(e)
    latest_time_pull = None

Latest pull from  2020-12-28 02:58:15.460000


In [16]:
split_utc_time_str('2020-11-13T21:32:20Z')

('2020-11-13', '17:32')

## Pull from Spotify and Store to DB 

In [17]:
newly_played = get_recently_played(latest_time_pull)

Getting songs at  2020-12-28 04:50:09.797292
There are 8 songs between  2020-12-28 04:26:28.873000   and   2020-12-28 04:49:05.134000


In [18]:
newly_played.head()

,name,artist_name,played_at_date,played_at_time,duration_min,popularity,song_uri,artist_id,playlist_id,after_ts
0,Please Come Home For Christmas,Adam Lambert,2020-12-28,00:49,2.964400,68,spotify:track:6PMhkA8o1fsn3fLCYuFp9r,spotify:artist:6prmLEyn4LfHlD9NnXWlf7,spotify:playlist:37i9dQZF1DX0A8zVl7p82B,1609130945134
1,One More Sleep,Leona Lewis,2020-12-28,00:46,3.993100,83,spotify:track:78pn8k7RogKo2oxl0DyX6d,spotify:artist:5lKZWd6HiSCLfnDGrq9RAm,spotify:playlist:37i9dQZF1DX0A8zVl7p82B,1609130945134
2,Let It Snow (with Babyface),Tori Kelly,2020-12-28,00:42,1.973233,79,spotify:track:3BhlC2eX8obANfmLEwo1ia,spotify:artist:1vSN1fsvrzpbttOYGsliDr,spotify:playlist:37i9dQZF1DX0A8zVl7p82B,1609130945134
3,I'll Be Home,Meghan Trainor,2020-12-28,00:40,3.660883,77,spotify:track:0lTurWECr2XL607eJAj1A3,spotify:artist:6JL8zeS1NmiOftqZTRgdTz,spotify:playlist:37i9dQZF1DX0A8zVl7p82B,1609130945134
4,I Hate the Holidays,Savannah Sgro,2020-12-28,00:36,2.696200,61,spotify:track:1pf77Bh4hFKA18cM5IRLko,spotify:artist:5aj9AKqFL0JpL2sQ8Q2irp,spotify:playlist:37i9dQZF1DX0A8zVl7p82B,1609130945134


In [19]:
con = sqlite3.connect(db_location)
newly_played.to_sql('Listening_History', con, if_exists='append' )

## Get Artist Info 

In [20]:
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()
tables = [x[0] for x in tables]
tables

['Listening_History', 'Artists_Info']

In [21]:
six_months_ago = datetime.datetime.now() - datetime.timedelta(days=6*30)
six_months_ago_str = str(six_months_ago.date())
sql_statement = f'select artist_name, artist_id, pull_date from artists_info where pull_date > {six_months_ago_str}'
sql_statement

'select artist_name, artist_id, pull_date from artists_info where pull_date > 2020-07-01'

In [22]:
con = sqlite3.connect(db_location)

existing_artists = pd.read_sql(sql_statement, con)
existing_artists['artist_id'] = 'spotify:artist:' + existing_artists['artist_id']
existing_artists = existing_artists.drop_duplicates(subset=['artist_id'])
existing_artists.head()

,artist_name,artist_id,pull_date
0,John Oates,spotify:artist:4TjR4K8rD5CqqqU59V6aGr,2020-12-15
1,Freddie King,spotify:artist:5dCuFngSPyOOnTAvrC7v2s,2020-12-15
2,Tony Bennett,spotify:artist:2lolQgalUvZDfp5vvVtTYV,2020-12-15
3,Khanipova,spotify:artist:7KHQppBj7ea6qRp4NsTN1u,2020-12-15
4,Bakar,spotify:artist:3K2Srho6NCF3o9MswGR76H,2020-12-15


In [23]:
existing_artists.shape

(888, 3)

In [24]:
unique_newly_played = newly_played[['artist_name','artist_id']].drop_duplicates(['artist_id'])
cnt_unique_newly_played = unique_newly_played.shape[0]
merged = pd.merge(unique_newly_played, existing_artists, on='artist_id', how='left', indicator='True')
new_artists = merged[merged['True'] == 'left_only']
new_artists_ids = new_artists['artist_id'].values.tolist()
new_artists

,artist_name_x,artist_id,artist_name_y,pull_date,True
0,Adam Lambert,spotify:artist:6prmLEyn4LfHlD9NnXWlf7,NaN,NaN,left_only
4,Savannah Sgro,spotify:artist:5aj9AKqFL0JpL2sQ8Q2irp,NaN,NaN,left_only
7,Miley Cyrus,spotify:artist:5YGY8feqx7naU7z4HrwZM6,NaN,NaN,left_only


In [25]:
def parse_artist_rslt(artist_dict):
    out = {}
    out['artist_name'] = artist_dict.get('name')
    out['artist_id'] = artist_dict.get('id')
    out['followers'] = artist_dict.get('followers').get('total')
    out['genres'] = str(artist_dict.get('genres'))
    out['popularity'] = artist_dict.get('popularity')
    out['pull_date'] = datetime.datetime.now().date()
    out_series = pd.Series(out)
    return(out_series)

In [27]:
if len(new_artists_ids) > 0:
    print("Pulling new artist info")
    artists_rslt = spotify.artists(new_artists_ids)
    artist_series = [parse_artist_rslt(x) for x in artists_rslt['artists']]
    artist_df = pd.concat(artist_series, axis=1).T
    artist_df.to_sql('Artists_Info', con, index=False, if_exists='append')


Pulling new artist info


In [28]:
con.commit()
con.close()

In [29]:
if 'artist_df' in locals():
    print(artist_df)

     artist_name               artist_id followers  \
0   Adam Lambert  6prmLEyn4LfHlD9NnXWlf7   1138052   
1  Savannah Sgro  5aj9AKqFL0JpL2sQ8Q2irp      5703   
2    Miley Cyrus  5YGY8feqx7naU7z4HrwZM6  13611231   

                                              genres popularity   pull_date  
0  ['dance pop', 'electropop', 'idol', 'pop', 'po...         68  2020-12-28  
1                                                 []         48  2020-12-28  
2  ['dance pop', 'pop', 'pop dance', 'post-teen p...         91  2020-12-28  
